In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import pandas as pd

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
from astropy.coordinates import SkyCoord
import astropy.units as u

import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

In [ ]:
# Doublet candidates
cat_doublet = Table.read('../catalogs/COLA1_O3doublet_catalog_1.0.fits').to_pandas()
cat_doublet['NUMBER'] = cat_doublet['NUMBER_1']
cat_doublet = cat_doublet.drop(columns=['NUMBER_1'])
# Singlet candidates
cat_singlet = Table.read('../catalogs/COLA1_O3_fitted_flux_singlet.fits').to_pandas()

print(cat_doublet.dtypes)
print(cat_singlet.dtypes)

In [ ]:
# Join singlet and doublet catalogs
# TMP supress cat_singlet
cat_singlet = cat_singlet[np.zeros(len(cat_singlet)).astype(bool)]
cat = cat_doublet.merge(cat_singlet, left_on='NUMBER', right_on='NUMBER',
                        how='outer', suffixes=['_d', '_s'])

def calculate_z(row):
    if pd.notna(row['z_O3_5008']) and pd.notna(row['z_O3doublet']):
        return (row['z_O3_5008'] + row['z_O3doublet']) * 0.5
    else:
        return row['z_O3_5008'] if pd.notna(row['z_O3_5008']) else row['z_O3doublet']

def calculate_RA(row):
    if pd.notna(row['RA']) and pd.notna(row['ALPHA_J2000_det']):
        return (row['RA'] + row['ALPHA_J2000_det']) * 0.5
    else:
        return row['RA'] if pd.notna(row['RA']) else row['ALPHA_J2000_det']

def calculate_DEC(row):
    if pd.notna(row['DEC']) and pd.notna(row['DELTA_J2000_det']):
        return (row['DEC'] + row['DELTA_J2000_det']) * 0.5
    else:
        return row['DEC'] if pd.notna(row['DEC']) else row['DELTA_J2000_det']

cat['z'] = cat.apply(calculate_z, axis=1)
cat['RA'] = cat.apply(calculate_RA, axis=1)
cat['DEC'] = cat.apply(calculate_DEC, axis=1)

cat['det_doublet'] = (cat['z_O3doublet'] > 0)
cat['det_singlet'] = (cat['z_O3_5008'] > 0)

cat.dtypes

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))

ax.hist(cat['z'], bins=60, histtype='stepfilled',
        edgecolor='k', facecolor='gray', lw=2)

ax.axvline(6.5916, c='r', ls='--', label='COLA1 redshift', zorder=-99)

ax.legend(fontsize=13)

ax.set_xticks(np.arange(5.25, 7.25, 0.1))

ax.set_xlabel('Redshift')
ax.set_ylabel('N objects')

# path_to_savefig = '/home/alberto/cosmos/ista/COLA1/paper/figures'
# fig.savefig(f'{path_to_savefig}/overdensities_hist.pdf', bbox_inches='tight',
#             pad_inches=0.1, facecolor='w')
plt.show()

fig, ax = plt.subplots()

zbins = np.linspace(6.45, 6.65, 40)
ax.hist(cat['z'], bins=zbins, label='all', alpha=0.9)
ax.hist(cat['z'][cat['det_doublet']], bins=zbins, label='doublet', histtype='step',
        lw=2)
ax.hist(cat['z'][cat['det_singlet']], bins=zbins, label='singlet', histtype='step',
        lw=2)
ax.set_xlabel('z', fontsize=20)

ax.axvline(6.5916)
ax.axvline(6.6116)
ax.axvline(6.5716)

# ax.legend()

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter('RA', 'DEC', data=cat[cat['det_doublet']],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
cbar = ax.scatter('RA', 'DEC', data=cat[cat['det_singlet']],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 9269],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

fig.colorbar(cbar)

ax.set_aspect('equal')

ax.set_xlim(150.6, 150.7)
ax.set_ylim(2.135, 2.27)

ax.set_xlabel('RA', fontsize=18)
ax.set_ylabel('DEC', fontsize=18)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

mask = (cat['z'] > 6.45) & (cat['z'] < 6.65)

z_min = cat['z'][mask].min()
z_max = cat['z'][mask].max()

ax.scatter('RA', 'DEC', data=cat[cat['det_doublet'] & mask],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
cbar = ax.scatter('RA', 'DEC', data=cat[cat['det_singlet'] & mask],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 9269],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# Distance at z=5.877
ang_diameter_dist = cosmo.angular_diameter_distance(5.877) / u.rad
print(ang_diameter_dist)
ang_diameter_dist = ang_diameter_dist.to(u.kpc / u.deg)
ang_per_dist = (300 * u.kpc / ang_diameter_dist)
ax.errorbar(150.67, 2.16, xerr=ang_per_dist.value * 0.5,
            fmt='none', capsize=4, c='k', elinewidth=2,
            capthick=2)
ax.text(150.67, 2.156, '300 pkpc',
        horizontalalignment='center')

fig.colorbar(cbar)
ax.set_title('6.45 < z < 6.65')

ax.set_aspect('equal')

ax.set_xlim(150.6, 150.7)
ax.set_ylim(2.135, 2.27)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))

mask = (cat['z'] > 5.877 - 0.077) & (cat['z'] < 5.877 + 0.077)

z_min = cat['z'][mask].min()
z_max = cat['z'][mask].max()

ax.scatter('RA', 'DEC', data=cat[cat['det_doublet'] & mask],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
cbar = ax.scatter('RA', 'DEC', data=cat[cat['det_singlet'] & mask],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1's clone
ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 9275],
           marker='*', s=300, c='z', cmap='gist_rainbow', vmin=z_min, vmax=z_max)
ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 6123],
           marker='*', s=300, c='z', cmap='gist_rainbow', vmin=z_min, vmax=z_max)
# ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 3590],
#            marker='*', s=300, c='z', cmap='gist_rainbow', vmin=z_min, vmax=z_max)
# ax.scatter('RA', 'DEC', data=cat[cat['NUMBER'] == 4382],
#            marker='*', s=300, c='z', cmap='gist_rainbow', vmin=z_min, vmax=z_max)

# Distance at z=5.877
ang_diameter_dist = cosmo.angular_diameter_distance(5.877) / u.rad
print(ang_diameter_dist)
ang_diameter_dist = ang_diameter_dist.to(u.kpc / u.deg)
ang_per_dist = (1000 * u.kpc / ang_diameter_dist)
ax.errorbar(150.67, 2.16, xerr=ang_per_dist.value * 0.5,
            fmt='none', capsize=4, c='k', elinewidth=2,
            capthick=2)
ax.text(150.67, 2.156, '1 pMpc',
        horizontalalignment='center')

fig.colorbar(cbar)
ax.set_title('5.65 < z < 5.95')

ax.set_aspect('equal')

ax.set_xlim(150.6, 150.7)
ax.set_ylim(2.135, 2.27)

plt.show()

In [ ]:
# Distance from each source to COLA1 in the comoving frame
COLA1_index = (cat['NUMBER'] == 9269)

cdist0 = cosmo.comoving_distance(cat['z'])
COLA1_dist = cosmo.comoving_distance(cat['z'][COLA1_index])

coord_Arr = SkyCoord(np.array(cat['RA']) * u.deg,
                     np.array(cat['DEC']) * u.deg,
                     distance=cdist0)
coord_COLA1 = SkyCoord(np.array(cat['RA'][COLA1_index]) * u.deg,
                       np.array(cat['DEC'][COLA1_index]) * u.deg,
                       distance=COLA1_dist)

dist_to_COLA1 = coord_COLA1.separation_3d(coord_Arr).to(u.Mpc)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

z_min = cat['z'].min()
z_max = cat['z'].max()

mask_bad_fit = (cat['f_O3_5008_err_d'] < 90.) & (cat['f_O3_4960_err_d'] < 90.)
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[mask_bad_fit],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
mask_bad_fit = (cat['f_O3_5008_err_s'] < 90.) & (cat['f_O3_4960_err_s'] < 90.)
cbar = ax.scatter('f_O3_5008_s', 'f_O3_4960_s', data=cat[mask],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 9269],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 9275],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 6123],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

ax.plot([0, 90], [0, 30], ls='--', c='k')

# TMP!!!!!!!!!!
ax.set_ylim(-1, 20)
ax.set_xlim(-1, 40)

fig.colorbar(cbar)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

mask = (cat['z'] > 6.45) & (cat['z'] < 6.65)

z_min = cat['z'][mask].min()
z_max = cat['z'][mask].max()

mask_bad_fit = (cat['f_O3_5008_err_d'] < 90.) & (cat['f_O3_4960_err_d'] < 90.) & mask
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[mask_bad_fit],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
mask_bad_fit = (cat['f_O3_5008_err_s'] < 90.) & (cat['f_O3_4960_err_s'] < 90.) & mask
cbar = ax.scatter('f_O3_5008_s', 'f_O3_4960_s', data=cat[mask],
                  c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 9269],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

ax.plot([0, 90], [0, 30], ls='--', c='k')

# TMP!!!!!!!!!!
ax.set_ylim(-1, 20)
ax.set_xlim(-1, 40)

ax.set_title('6.45 < z < 6.65')

fig.colorbar(cbar)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))

mask = (cat['z'] > 5.877 - 0.077) & (cat['z'] < 5.877 + 0.077)

z_min = cat['z'][mask].min()
z_max = cat['z'][mask].max()

mask_bad_fit = (cat['f_O3_5008_err_d'] < 90.) & (cat['f_O3_4960_err_d'] < 90.) & mask
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[mask_bad_fit],
           c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

# COLA1
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 9275],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)
ax.scatter('f_O3_5008_d', 'f_O3_4960_d', data=cat[cat['NUMBER'] == 6123],
           marker='*', s=300, c='z', cmap='rainbow', vmin=z_min, vmax=z_max)

ax.plot([0, 90], [0, 30], ls='--', c='k')

# TMP!!!!!!!!!!
ax.set_ylim(-1, 20)
ax.set_xlim(-1, 40)

ax.set_title('5.80 < z < 5.95')

fig.colorbar(cbar)

plt.show()

In [ ]:
Hbeta_SN = cat['f_Hb_d'] / cat['f_Hb_err_d']
O3_5008_SN = cat['f_O3_5008_d'] / cat['f_O3_5008_err_d']
mask_Hbeta_SN = (Hbeta_SN > 4) & (O3_5008_SN > 5) & (cat['F115W_AUTO_mag'] < 90)


fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

cbar = ax.scatter(cat['F115W_AUTO_mag'][mask_Hbeta_SN],
                 (cat['f_O3_5008_d'] / cat['f_Hb_d'])[mask_Hbeta_SN],
                 c=cat['z'][mask_Hbeta_SN], cmap='rainbow', vmin=z_min, vmax=z_max)
ax.scatter(cat['F115W_AUTO_mag'][cat['NUMBER'] == 9269],
           (cat['f_O3_5008_d'] / cat['f_Hb_d'])[cat['NUMBER'] == 9269],
           marker='*', s=200,
           c=cat['z'][cat['NUMBER'] == 9269], cmap='rainbow', vmin=z_min, vmax=z_max)

fig.colorbar(cbar)

ax.set_title('O3_5008 / Hbeta ratio')

ax.set_xlabel('F115W [mag]')
ax.set_ylabel('f_O3_5008 / f_Hbeta')

plt.show()

In [ ]:
fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.errorbar('f_O3_5008_d', 'f_Hb_d',
            xerr='f_O3_5008_err_d',
            yerr='f_Hb_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k')

ax.errorbar('f_O3_5008_d', 'f_Hb_d',
            xerr='f_O3_5008_err_d',
            yerr='f_Hb_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15)

ax.set_xlabel('f_O3_5008')
ax.set_ylabel('f_Hbeta')

plt.show()

In [ ]:
fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.errorbar('F115W_AUTO_mag', 'f_Hb_d',
            yerr='f_Hb_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k')
ax.errorbar('F115W_AUTO_mag', 'f_Hb_d',
            yerr='f_Hb_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15)

ax.set_xlabel('F115W')
ax.set_ylabel('f_Hbeta')

plt.show()

fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.errorbar('F115W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k')
ax.errorbar('F115W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15)

ax.set_xlabel('F115W')
ax.set_ylabel('f_O3_5008')

plt.show()

In [ ]:
# Load eiger catalogs
eiger_cat_list = ['J0100_photcat_v4_O3CANDIDATES_all_CLUMPS.fits',  'J1030_photcat_v1_O3CANDIDATES_all_CLUMPS.fits', 
                  'J1148_photcat_v2_O3CANDIDATES_all_CLUMPS.fits', 'J0148_photcat_v1_O3CANDIDATES_all_CLUMPS.fits', 
                  'J1120_photcat_v2_O3CANDIDATES_all_CLUMPS.fits']
eiger_cat_dir = '/home/alberto/cosmos/ista/COLA1/catalogs/eiger'


tab_list = []

for cat_name in eiger_cat_list:
    tab_list.append(Table.read(f'{eiger_cat_dir}/{cat_name}').to_pandas())

eiger_cat = pd.concat(tab_list)

In [ ]:
fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter('f_O3_5008', 'f_Hb', data=eiger_cat, s=2,
           label='EIGER')

ax.errorbar('f_O3_5008_d', 'f_Hb_d',
            xerr='f_O3_5008_err_d',
            yerr='f_Hb_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k', label='COLA1 (all sources)')

ax.errorbar('f_O3_5008_d', 'f_Hb_d',
            xerr='f_O3_5008_err_d',
            yerr='f_Hb_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15, label='COLA1')

ax.legend()

ax.set_xlabel('f_O3_5008')
ax.set_ylabel('f_Hbeta')

ax.set_ylim(-1, 10)
ax.set_xlim(-1, 40)

plt.show()

In [ ]:
fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter(eiger_cat['MAG_AUTO_F115W_apcor'], eiger_cat['f_Hb'], s=2,
           label='EIGER')

ax.errorbar('F115W_AUTO_mag', 'f_Hb_d',
            yerr='f_Hb_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k', label='COLA1 (all sources)')
ax.errorbar('F115W_AUTO_mag', 'f_Hb_d',
            yerr='f_Hb_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15, label='COLA1')

ax.legend()

ax.set_xlabel('F115W')
ax.set_ylabel('f_Hbeta')

ax.set_ylim(-1, 10)
ax.set_xlim(23, 30.5)

plt.show()

fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter(eiger_cat['MAG_AUTO_F115W_apcor'], eiger_cat['f_O3_5008'], s=2,
           label='EIGER')

ax.errorbar('F115W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[mask_Hbeta_SN], ls='', fmt='o',
            c='k', label='COLA1 (all sources)')
ax.errorbar('F115W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15, label='COLA1')

ax.set_xlabel('F115W')
ax.set_ylabel('f_O3_5008')

ax.legend()

ax.set_ylim(-1, 40)
ax.set_xlim(23, 30.5)

plt.show()

fig, ax = plt.subplots()

z_min = cat['z'].min()
z_max = cat['z'].max()

ax.scatter(eiger_cat['MAG_AUTO_F356W_apcor'], eiger_cat['f_O3_5008'], s=2,
           label='EIGER')

ax.errorbar('F356W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat, ls='', fmt='o',
            c='k', label='COLA1 (all sources)')
ax.errorbar('F356W_AUTO_mag', 'f_O3_5008_d',
            yerr='f_O3_5008_err_d',
            data=cat[cat['NUMBER'] == 9269], ls='', fmt='*',
            c='r', ms=15, label='COLA1')

ax.set_xlabel('F356W')
ax.set_ylabel('f_O3_5008')

ax.legend()

ax.set_ylim(-1, 40)
ax.set_xlim(23, 30.5)

plt.show()

In [ ]:
# Load COLA1 fluxes
COLA1_fluxes = Table.read('COLA1_fluxes/COLA1_O3_fitted_flux.fits')
COLA1_fluxes = COLA1_fluxes[COLA1_fluxes['NUMBER_1'] == 9269]
COLA1_fluxes.keys()

In [ ]:
fig, ax = plt.subplots()

wav_list = [4341, 4364, 4862, 4960, 5008]
for i, suffix in enumerate(['_A', '_B', '']):
    flx_list = np.array([COLA1_fluxes[f'f_Hg{suffix}'], COLA1_fluxes[f'f_O3_4363{suffix}'], COLA1_fluxes[f'f_Hb{suffix}'],
                COLA1_fluxes[f'f_O3_4960{suffix}'], COLA1_fluxes[f'f_O3_5008{suffix}']]).flatten()
    flx_err_list = np.array([COLA1_fluxes[f'f_Hg{suffix}_err'], COLA1_fluxes[f'f_O3_4363{suffix}_err'], COLA1_fluxes[f'f_Hb{suffix}_err'],
                    COLA1_fluxes[f'f_O3_4960{suffix}_err'], COLA1_fluxes[f'f_O3_5008{suffix}_err']]).flatten()

    ax.errorbar(np.array(wav_list) + 4*i, flx_list,
                yerr=flx_err_list,
                fmt='o')

ax.set_yscale('log')
ax.set_xlabel('Wavelength')
ax.set_ylabel('Flux')
ax.set_ylim(0.1, 100)

plt.show()

In [ ]:
Hg_A = COLA1_fluxes['f_Hg_A'][0]
Hg_B = COLA1_fluxes['f_Hg_B'][0]

Hb_A = COLA1_fluxes['f_Hb_A'][0]
Hb_B = COLA1_fluxes['f_Hb_B'][0]

O3_4364_A = COLA1_fluxes['f_O3_4363_A'][0]
O3_4364_B = COLA1_fluxes['f_O3_4363_B'][0]
O3_4364 = COLA1_fluxes['f_O3_4363'][0]

O3_4364_A_err = COLA1_fluxes['f_O3_4363_A_err'][0]
O3_4364_B_err = COLA1_fluxes['f_O3_4363_B_err'][0]
O3_4364_err = COLA1_fluxes['f_O3_4363_err'][0]

O3_5008_A = COLA1_fluxes['f_O3_5008_A'][0]
O3_5008_B = COLA1_fluxes['f_O3_5008_B'][0]
O3_5008 = COLA1_fluxes['f_O3_5008'][0]

O3_4960_A = COLA1_fluxes['f_O3_4960_A'][0]
O3_4960_B = COLA1_fluxes['f_O3_4960_B'][0]
O3_4960 = COLA1_fluxes['f_O3_4960'][0]

O3_5008_A_err = COLA1_fluxes['f_O3_5008_A_err'][0]
O3_5008_B_err = COLA1_fluxes['f_O3_5008_B_err'][0]
O3_5008_err = COLA1_fluxes['f_O3_5008_err'][0]

O3_4960_A_err = COLA1_fluxes['f_O3_4960_A_err'][0]
O3_4960_B_err = COLA1_fluxes['f_O3_4960_B_err'][0]
O3_4960_err = COLA1_fluxes['f_O3_4960_err'][0]

Hg_A_err = COLA1_fluxes['f_Hg_A_err'][0]
Hg_B_err = COLA1_fluxes['f_Hg_B_err'][0]

Hb_A_err = COLA1_fluxes['f_Hb_A_err'][0]
Hb_B_err = COLA1_fluxes['f_Hb_B_err'][0]

Hb = COLA1_fluxes['f_Hb'][0]
Hg = COLA1_fluxes['f_Hg'][0]
Hb_err = COLA1_fluxes['f_Hb_err'][0]
Hg_err = COLA1_fluxes['f_Hg_err'][0]

print(f'S/N modA')
print(f'O3_5008 = {O3_5008_A / O3_5008_A_err:0.2f}')
print(f'O3_4960 = {O3_4960_A / O3_4960_A_err:0.2f}')
print(f'Hb = {Hb_A / Hb_A_err:0.2f}')
print(f'O3_4363 = {O3_4364_A / O3_4364_A_err:0.2f}')
print(f'Hg = {Hg_A / Hg_A_err:0.2f}')
print()
print(f'S/N modB')
print(f'O3_5008 = {O3_5008_B / O3_5008_B_err:0.2f}')
print(f'O3_4960 = {O3_4960_B / O3_4960_B_err:0.2f}')
print(f'Hb = {Hb_B / Hb_B_err:0.2f}')
print(f'O3_4363 = {O3_4364_B / O3_4364_B_err:0.2f}')
print(f'Hg = {Hg_B / Hg_B_err:0.2f}')

print()

print(f'A: Hgamma = {Hg_A:0.3f} ± {Hg_A_err:0.3f}')
print(f'B: Hgamma = {Hg_B:0.3f} ± {Hg_B_err:0.3f}')
print()
print(f'A: Hbeta = {Hb_A:0.3f} ± {Hb_A_err:0.3f}')
print(f'B: Hbeta = {Hb_B:0.3f} ± {Hb_B_err:0.3f}')
print()
print(f'Combined: Hgamma = {Hg:0.3f} ± {Hg_err:0.3f}')
print(f'Combined: Hbeta = {Hb:0.3f} ± {Hb_err:0.3f}')

Hg_to_Hb = Hg / Hb
Hg_to_Hb_err = ((Hg_err / Hb)**2 + (Hg/(Hb**2) * Hb_err)**2)**0.5

print()
print(f'Combined: Hg/Hb = {Hg_to_Hb:0.3f} ± {Hg_to_Hb_err:0.3f}')

print()
print('--------------')
print()

print(f'A: O3_4364 = {O3_4364_A:0.3f} ± {O3_4364_A_err:0.3f}')
print(f'B: O3_4364 = {O3_4364_B:0.3f} ± {O3_4364_B_err:0.3f}')
print()
print(f'A: O3_5008 = {O3_5008_A:0.3f} ± {O3_5008_A_err:0.3f}')
print(f'B: O3_5008 = {O3_5008_B:0.3f} ± {O3_5008_B_err:0.3f}')
print(f'A: O3_4960 = {O3_4960_A:0.3f} ± {O3_4960_A_err:0.3f}')
print(f'B: O3_4960 = {O3_4960_B:0.3f} ± {O3_4960_B_err:0.3f}')
print()
print(f'Combined: O3_4364 = {O3_4364:0.3f} ± {O3_4364_err:0.3f}')
print(f'Combined: O3_5008 = {O3_5008:0.3f} ± {O3_5008_err:0.3f}')

O3_4364_to_5008 = O3_4364 / O3_5008
O3_4364_to_5008_err = ((O3_4364_err / O3_5008)**2 + (O3_4364/(O3_5008**2) * O3_5008_err)**2)**0.5

print()
print(f'Combined: O3_4364/O3_5008 = {O3_4364_to_5008:0.3f} ± {O3_4364_to_5008_err:0.3f}')

In [ ]:
both_modules_mask = (cat['z_O3doublet_A_n'] > 0) & (cat['z_O3doublet_B_n'] > 0)
print(sum(both_modules_mask))
only_A_mask = (cat['z_O3doublet_A_n'] > 0) & ~(cat['z_O3doublet_B_n'] > 0)
only_B_mask = ~(cat['z_O3doublet_A_n'] > 0) & (cat['z_O3doublet_B_n'] > 0)

fig, ax = plt.subplots()

ax.errorbar(cat['z_O3doublet_A_n'][both_modules_mask],
           cat['z_O3doublet_A_n'][both_modules_mask] - cat['z_O3doublet_B_n'][both_modules_mask],
           yerr=(cat['z_O3doublet_A_err_n'][both_modules_mask]**2 - cat['z_O3doublet_B_err_n'][both_modules_mask]**2)**0.5,
           fmt='o')

ax.scatter(cat['z_O3doublet_A_n'][cat['NUMBER'] == 9269],
           cat['z_O3doublet_A_n'][cat['NUMBER'] == 9269] - cat['z_O3doublet_B_n'][cat['NUMBER'] == 9269],
           marker='*', s=200, color='r', zorder=99)

ax.axhline(0, ls='-', c='k', zorder=-99)
c = 299792.458
mean_offset = np.average(cat['z_O3doublet_A_n'][both_modules_mask] - cat['z_O3doublet_B_n'][both_modules_mask])
std_offset = np.std(cat['z_O3doublet_A_n'][both_modules_mask] - cat['z_O3doublet_B_n'][both_modules_mask])
print(f'{mean_offset=}, {std_offset=}')
ax.axhline(mean_offset, ls='--', c='dimgray')

ax.set_ylim(-0.0075, 0.0075)

ax.set_ylabel('zA - zB', fontsize=15)
ax.set_xlabel('zA', fontsize=15)

plt.show()

In [ ]:
# Balmer decrement
N_iter = 10000
this_Hb_Arr = np.random.normal(Hb, Hb_err, size=N_iter)
this_Hg_Arr = np.random.normal(Hg, Hg_err, size=N_iter)

balmer_dec = np.mean(this_Hb_Arr * 0.47 / this_Hg_Arr)
balmer_dec_err = np.std(this_Hb_Arr * 0.47 / this_Hg_Arr)

print(f'Balmer decrement: {balmer_dec:0.4f} +/- {balmer_dec_err:0.4f}')

In [ ]:
# Compute SFR
# For now we assume the same value of xi_ion as in Matthee2023.
SFR_C = 41.59
N_iter = 10_000

mag_ext_corr = 0.003

Dl = cosmo.luminosity_distance(6.59165).to(u.cm).value

Hb_L_Arr = np.random.normal(Hb * (1 + mag_ext_corr), Hb_err, size=N_iter) * 1e-18 * 4 * np.pi * Dl**2
balmer_dec_Arr = np.random.normal(balmer_dec, balmer_dec_err, size=N_iter)

SFR_Arr = 2.86 * Hb_L_Arr / balmer_dec_Arr / 10 ** SFR_C
SFR, SFR_up, SFR_down = np.percentile(SFR_Arr, [50, 84, 16])
print(f'SFR = {SFR:0.2f} + {SFR_up - SFR:0.2f} - {SFR - SFR_down:0.2f}')

In [ ]:
N_iter = 100000

area = (4 * np.pi * cosmo.luminosity_distance(6.591).to(u.cm).value**2)

mag_ext_corr = 0.003

this_Lya_M18 = np.random.normal(4.1, 0.2, size=N_iter)
this_Lya_M18 = this_Lya_M18 * 1e43 / area * 1e18

this_Hb = np.random.normal(Hb * (1 + mag_ext_corr), Hb_err, size=N_iter)

this_balmer_dec = np.random.normal(balmer_dec, balmer_dec_err, size=N_iter)

fesc_list = this_Lya_M18 / (8.7 * 2.86 * this_Hb / this_balmer_dec)

fesc_Lya, fesc_Lya_down, fesc_Lya_up = np.percentile(fesc_list, [50, 16, 84])

print(f'f_esc(Lya) = {fesc_Lya:0.3f} + {fesc_Lya_up - fesc_Lya:0.3f} - {fesc_Lya - fesc_Lya_down:0.3f}')

In [ ]:
from astropy.io import fits

cat_indep_z = fits.open('z_calibration/COLA1_O3_indep_redshift.fits')[1]

c = 299792.458 # km/s
dz_A = cat_indep_z.data['dz_A'] * c
dz_B = cat_indep_z.data['dz_B'] * c

mask_both_modules = (dz_A != 0) & (dz_B != 0)

fig, ax = plt.subplots()

# mask = (dz_A > 0) & (cat_indep_z.data['F115W_AUTO_mag'] < 30)
# ax.scatter(cat_indep_z.data['F115W_AUTO_mag'][mask], dz_A[mask])
# mask = (dz_B > 0) & (cat_indep_z.data['F115W_AUTO_mag'] < 30)
# ax.scatter(cat_indep_z.data['F115W_AUTO_mag'][mask], dz_B[mask])

mask = mask_both_modules & (cat_indep_z.data['F115W_AUTO_mag'] < 29)
ax.scatter(cat_indep_z.data['F115W_AUTO_mag'][mask], dz_B[mask], label='modB', c='C1')
ax.scatter(cat_indep_z.data['F115W_AUTO_mag'][mask], dz_A[mask], label='modA', c='C0')
ax.axhline(np.median(dz_A[mask]), c='C0', ls=':')
ax.axhline(np.median(dz_B[mask]), c='C1', ls=':')
print(np.median(dz_A[mask]), np.median(dz_B[mask]))

mask = cat_indep_z.data['NUMBER_1'] == 9269
print(f' COLA1 ModA: dv = {dz_A[mask][0]:0.3f}')
print(f' COLA1 ModB: dv = {dz_B[mask][0]:0.3f}')
print(f' COLA1 ModA: dz = {dz_A[mask][0] / c:0.6f}')
print(f' COLA1 ModB: dz = {dz_B[mask][0] / c:0.6f}')

ax.legend()

ax.axhline(0, ls='--', c='k')

ax.set_xlabel('F115W [magAB]')
ax.set_ylabel(r'$\Delta z$ [km\,s$^{-1}$]')

plt.show()

In [ ]:
z_O3_5008_A = cat_indep_z.data['z_O3doublet_A_n'][mask][0]
z_O3_5008_B = cat_indep_z.data['z_O3doublet_B_n'][mask][0]
dz_A = cat_indep_z.data['dz_A'][mask][0]
dz_B = cat_indep_z.data['dz_B'][mask][0]
z_Hbeta_A = 6.59166189
z_Hbeta_B = 6.58978584
z_Hgamma_A = 6.59300324
z_Hgamma_B = 6.58985508

print('ModA')
mask = cat_indep_z.data['NUMBER_1'] == 9269
print(f'z_O3_5008 = {z_O3_5008_A:0.6f}')
print(f'z_O3_4960 = {z_O3_5008_A + dz_A:0.6f}')
print(f'z_Hbeta = {z_Hbeta_A:0.6f}')
print(f'z_Hgamma = {z_Hgamma_A:0.6f}')

print()

print('ModB')
print(f'z_O3_5008 = {z_O3_5008_B:0.6f}')
print(f'z_O3_4960 = {z_O3_5008_B + dz_B:0.6f}')
print(f'z_Hbeta = {z_Hbeta_B:0.6f}')
print(f'z_Hgamma = {z_Hgamma_B:0.6f}')

In [ ]:
# Load Lya spectrum
hdul = fits.open('../spectra/1D_COLA1_2020_v2_newest.fits')
cola1_UV_spec = hdul[1].data

In [ ]:
fig, ax = plt.subplots()

w_lya = 1215.67

wav_range = (cola1_UV_spec['vacuum_wavelength'] < w_lya * 7.6 + 10) & (cola1_UV_spec['vacuum_wavelength'] > w_lya * 7.6 - 25)

ax.plot(cola1_UV_spec['vacuum_wavelength'][wav_range], cola1_UV_spec['fluxdensity'][wav_range],
        c='dimgray')


ax.axvline(w_lya * (1 + z_O3_5008_A), c='C0', label='O3_5008', lw=2)
ax.axvline(w_lya * (z_O3_5008_A + dz_A + 1), c='C1', label='O3_4960', lw=2)
ax.axvline(w_lya * (1 + z_Hbeta_A), c='C2', label='Hbeta', lw=2)
ax.axvline(w_lya * (1 + z_Hgamma_A), c='C3', label='Hgamma', lw=2)

ax.legend()

ax.set_title('modA')

ax.set_xlabel(r'Wavelength [\AA]')

plt.show()

fig, ax = plt.subplots()

ax.plot(cola1_UV_spec['vacuum_wavelength'][wav_range], cola1_UV_spec['fluxdensity'][wav_range],
        c='dimgray')

ax.axvline(w_lya * (1 + z_O3_5008_B), c='C0', label='O3_5008', lw=2)
ax.axvline(w_lya * (z_O3_5008_B + dz_B + 1), c='C1', label='O3_4960', lw=2)
ax.axvline(w_lya * (1 + z_Hbeta_B), c='C2', label='Hbeta', lw=2)
ax.axvline(w_lya * (1 + z_Hgamma_B), c='C3', label='Hgamma', lw=2)

ax.legend()

ax.set_title('modB')

ax.set_xlabel(r'Wavelength [\AA]')

plt.show()

In [ ]:
# fig, ax = plt.subplots()

# mask = mask_both_modules
# bins = np.linspace(-5000, 5000, 60)

# ax.hist(dz_A[mask], bins, histtype='step', label='modA')
# ax.hist(dz_B[mask], bins, histtype='step', label='modB')

# ax.axvline(0, ls='--', c='k')

# ax.set_xlabel(r'$\Delta z$ [km\,s$^{-1}$]')

# ax.legend()

# plt.show()

# ##################################

# fig, ax = plt.subplots()

# mask = mask_both_modules
# bins = np.linspace(-5000, 5000, 60)

# mask = (dz_A != 0) & (cat_indep_z.data['F115W_AUTO_mag'] < 30)
# ax.hist(dz_A[mask], bins, histtype='step', label='Only modA', color='C0')

# mask = (dz_B != 0) & (cat_indep_z.data['F115W_AUTO_mag'] < 30)
# ax.hist(dz_B[mask], bins, histtype='step', label='Only modB', color='C1')

# ax.axvline(0, ls='--', c='k')

# ax.set_xlabel(r'$\Delta z$ [km\,s$^{-1}$]')

# ax.legend()

# plt.show()

In [ ]:
mask = cat_indep_z.data['NUMBER_1'] == 9269
z_A = cat_indep_z.data['z_O3doublet_A_n'][mask][0]
z_B = cat_indep_z.data['z_O3doublet_B_n'][mask][0]
dz_A =  cat_indep_z.data['dz_A'][mask][0]
dz_B =  cat_indep_z.data['dz_B'][mask][0]
w_O3_5008 = 5008.24
w_O3_4960 = 4960.295

z0_A = z_A
z0_B = z_B 

print(dz_A, dz_B)
print(f'z_real A: {z0_A}')
print(f'z_real B: {z0_B}')

In [ ]:
import pickle

# Model
def gaussian_O3_withfudge(x, a, c, redshift, sigma, fudge):
    x0_O31 = (1+redshift)*4960.295
    x0_O32 = (1+redshift)*5008.24
    return c + a*(np.exp(-(x-x0_O31)**2/(2*sigma**2))+c + 2.98*fudge*np.exp(-(x-x0_O32)**2/(2*sigma**2)))

w_rest_xx =  np.linspace(4900, 5100, 100)
res_yy_A = np.zeros_like(w_rest_xx).astype(float)
res_yy_B = np.zeros_like(w_rest_xx).astype(float)

mod_count_A = np.ones_like(res_yy_A)
mod_count_B = np.ones_like(res_yy_B)

SAVE_FOLDER = '../spectra/SPECTRA_O3_FINAL/REDSHIFT_FIT/'
FIELD = 'COLA1'

# Load the fits
for src in range(len(cat)):
    NclumpsY = cat['Nclumps_Y_d'][src]
    thisID = int(cat['NUMBER'][src])

    # [modA, modB] = True or False
    which_modules = [cat['f_O3_5008_A_d'][src] > 0,
                     cat['f_O3_5008_B_d'][src] > 0]

    for i in range(2): # For both modules
        if which_modules[i] and i == 0:
            module = 'A'
        elif which_modules[i] and i == 1:
            module = 'B'
        else:
            continue

        this_z = cat[f'z_O3doublet_{module}_n'][src]

        res_save_name = SAVE_FOLDER+'redshift_fit_residuals_%s_%s_mod%s.pkl' % (FIELD, thisID, module)

        with open(res_save_name, 'rb') as file:
            model_res = pickle.load(file)

        xx_obs = model_res['xx']
        model_res_obs = model_res['residuals']

        mask_nan = np.isfinite(model_res_obs)

        spec_bin_width = xx_obs[1] - xx_obs[0]
        lum_dist = cosmo.luminosity_distance(this_z).to(u.cm).value
        model_to_interp = model_res_obs[mask_nan] / spec_bin_width / (4 * np.pi * lum_dist **2)

        res_rest = np.interp(w_rest_xx, xx_obs[mask_nan] / (this_z + 1), model_to_interp,
                             left=np.nan, right=np.nan)

        mask_nan_rest = np.isfinite(res_rest)

        if module == 'A':
            res_yy_A[mask_nan_rest] += res_rest[mask_nan_rest]
            mod_count_A += mask_nan_rest.astype(int)
        elif module == 'B':
            res_yy_B[mask_nan_rest] += res_rest[mask_nan_rest]
            mod_count_B += mask_nan_rest.astype(int)

res_yy_A /= mod_count_A
res_yy_B /= mod_count_B

In [ ]:
fig, ax = plt.subplots()

ax.plot(w_rest_xx, res_yy_A, label='modA')
ax.plot(w_rest_xx, res_yy_B, label='modB')
ax.axvline(4960, c='r', ls='--')
ax.axvline(5008, c='r', ls='--')

ax.set_ylabel(r'Fit residuals [erg\,s$^{-1}$\AA$^{-1}$]')
ax.set_xlabel(r'$\lambda_0$ [\AA]')

ax.legend()

plt.show()